In [1]:
import pandas as pd
from keybert import KeyBERT
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel

/opt/homebrew/anaconda3/envs/capstone/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("menu_token_setnece_sample.csv")
# data=data.drop(columns=['Unnamed: 0'])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   rst_name               40 non-null     object
 1   review_sentence_split  40 non-null     object
 2   menu_name_split        40 non-null     object
 3   menu_sentence          40 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [4]:
import ast

# 문자열 형태의 리스트를 파이썬 리스트로 변환
data["menu_sentence"] = data["menu_sentence"].apply(ast.literal_eval)
# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인
data["menu_sentence"].iloc[0]

['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면서 자주 봤던 곳인데 서비스 메뉴도 알차고 좋네요',
 '매운 거 좋아   .',
 '아이들이랑 가서 반반 족발 먹었어요',
 '지점마다 서비스 내용이 다른가 봅니다',
 '일단 콩나물국 정말 맛있었어요',
 '서비스 5개 중에 막국수와 주먹밥 중에 엄청 고민했는데 아이들 위해서 주먹밥으로 결정했습니다',
 '반찬들이 정갈하고 하나같이 맛이 좋아요',
 '족발은 냄새 하나도 안나고 부드럽습니다',
 '아이들은 기본 족발을 먹고 저는 매운 족발을 상추와 알배추에 싸 먹었어요',
 '정말 맛있게 잘 먹었습니다  .',
 '양도 푸짐하고  곁들여 먹을 수 있는 각종 김치랑 따뜻한 국물에 서비스 메뉴까지 나와서 좋아요.',
 '구의동 먹자',
 '골목에서 롱런하는 이유가 있죠',
 '맨날 배달로만 먹다가 매장에서 먹으니 더 꿀맛입니다  윤기가 좔좔 흐르는거 보이시죠',
 '체인이라고 다 같은 체인은 아니잖아요',
 '친절한 사장님 덕에 맛있고 편안하게 잘 먹고왔어요',
 '매장 안 여기저기서 반반족발을 드시던데 진짜 둘 다 맛있어서 놓치고 싶지않아요',
 '한톨도 남기지 않고 순삭하고는 터지는 배를 부여잡고 나왔네요 ㅎㅎ.',
 '퇴근하고 현지인은 약속이라도 한듯 이 집으로 집결한다',
 '왜이리 적게 받으시지 싶을정도로 음식에 진심인 집  오랫동안 텃세높은 광진구에서 살아남은 비결이 고스란히 느껴지는 집  1  보쌈 버너 먹는 동안 따뜻하고 촉촉한 고기를 맛 볼 수 있다',
 '2  김치 보쌈 집의 근본인 김치가 무려 10종  김치를 먹기 위해 보쌈을 시킬 정도  3  서비스 따뜻한 콩나물국으로 입맛 돋우고 생굴 된찌 순두부찌개 막국수 주먹밥 중 취향껏 선택 4  푸짐한 양 둘이가서 푸파하려다가 실패하긴 처음 .',
 '광진구족발원탑답게 대존맛임',
 '역시 족발은 가족     넘넘 맛있게 먹었어요',
 '서비스로 선택한 얼큰순두부찌개도 대존맛.',
 '이름그대로 가장 맛있는 족발 

In [5]:
# 문자열 형태의 리스트를 파이썬 리스트로 변환
data["menu_name_split"] = data["menu_name_split"].apply(ast.literal_eval)
# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인
data["menu_name_split"].iloc[0]

['',
 '왕족',
 '알뜰',
 ')',
 '불족발',
 '실속',
 '보쌈',
 '싱글족발',
 '1',
 '발',
 '싱글보쌈',
 '어리굴젓',
 '통영',
 '불',
 '인',
 '하',
 'g',
 '왕족발',
 '야채',
 '새우젓',
 '알',
 '1인',
 '마늘',
 '수제',
 '싱글불족발',
 '실속알뜰보쌈',
 '족발하고',
 '쌈야채',
 '양념',
 '막국수)',
 '200',
 '알배추',
 '고',
 '족발',
 '수제양념막국수',
 '싱글',
 '200g',
 '쌈장',
 '막국수',
 '겉절이',
 '배추',
 '쌈']

In [6]:
data = data[["rst_name", "menu_sentence", "menu_name_split"]]
data

,rst_name,menu_sentence,menu_name_split
0,가장맛있는족발 구의역점,[잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면...,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어..."
1,강나루 유황오리주물럭 본점,"[생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ, 미나리랑 같이 먹는 오리주물...","[탕, 양념, 양념주물럭, 오리주물럭, 오리, 주물럭, 마리, 모듬, 훈제, 뼈탕,..."
2,고공 구의점,"[구의역 소고기 맛집입니다, 토시살 갈비살 버 섯구이 구성의 소고기 세트 주...","[, 돼지세트, 쫄면, 목살, 고공, 김치찌개, 갈비살, 밥, 된장찌개, 겹, 갈비..."
3,고기반햄반김치찌개&김치찜 아차산본점,"[보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요, 김치찌개 넘 좋...","[4인, 김치찌개, 불고기, 1, 반, 간장제육, 제육, 인, 세트, 고기반, 2,..."
4,고향집,"[새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음, 새조개 진짜 비싼데 양 많이...","[회, 구이, 새꼬막, 가자미, 참꼬막양념, 생태찌개, 꼬막, 간장게장, 재미, 간..."
5,동대문곱창,[소금막창 너무너무 맛있어서 어이없어요 주문 왕창해 놓고 이걸 어떻게 다 먹나 싶은...,"[탕, 알곱창, 야채곱창, 치즈곱창, 계란, 순대, 곱창, 치즈계란말이, 야채, 알..."
6,라라면옥,"[냉수육 미나리 무침이 모듬 수육 전골과 잘 어울리네요, 얼음맥주 처음인데 완전 시...","[, 냉면, 쌀국수, 양지, LA, 구이, 국밥, 냉수육과, 육, 비빔냉면, 수육전..."
7,라비올리,"[양은 많지 않아도 적당하고 좋아요., 아이랑 방문하기 너무 좋아요, 가성비도 괜...","[목살, 음료2, 치즈오븐스파게티, 불고기, 리조또, 아, 불고기로제, 의, 풍기빠..."
8,먹거리집,"[포장해와서 집밥이랑 같이 먹었어요, 역시 순대국 먹거리집 역시 맛있어요 둔둔하...","[감투, 수육, 오소리감투, 오소리, 순대국, 대, 밥, 중, 순대국밥, 편육]"
9,명가옥,[구의동 명가옥 옥 관 이렇게 끝나는 식당들은 전 통과 노하우가 있어 굉장히...,"[냉면, 곰탕, 도가니수육, 갈비찜, 모듬수육, /, 모듬, 도가니, 왕만두, 곱창..."


In [7]:
# Exploding the 'review' column to separate rows
df_exploded = data.explode("menu_sentence")

# Resetting index to make sure it's clean after exploding
df_exploded.reset_index(drop=True, inplace=True)

df_exploded

,rst_name,menu_sentence,menu_name_split
0,가장맛있는족발 구의역점,잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면서...,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어..."
1,가장맛있는족발 구의역점,매운 거 좋아 .,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어..."
2,가장맛있는족발 구의역점,아이들이랑 가서 반반 족발 먹었어요,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어..."
3,가장맛있는족발 구의역점,지점마다 서비스 내용이 다른가 봅니다,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어..."
4,가장맛있는족발 구의역점,일단 콩나물국 정말 맛있었어요,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어..."
...,...,...,...
9225,황실짜장,마파두부덮밥 조금 맵지만 맛있어요 .,"[물만두, 만, 불짜장, 삼선해물탕, 삼선간짜장, 해물탕, 꽃, 울, 꽃빵튀김, 울..."
9226,황실짜장,새우볶음밥 먹었는데 맛있었어요 .,"[물만두, 만, 불짜장, 삼선해물탕, 삼선간짜장, 해물탕, 꽃, 울, 꽃빵튀김, 울..."
9227,황실짜장,쟁반짜장 맛있고 좋았습니다.,"[물만두, 만, 불짜장, 삼선해물탕, 삼선간짜장, 해물탕, 꽃, 울, 꽃빵튀김, 울..."
9228,황실짜장,볶음밥 맛남ㅎ.,"[물만두, 만, 불짜장, 삼선해물탕, 삼선간짜장, 해물탕, 꽃, 울, 꽃빵튀김, 울..."


In [9]:
def find_menu_in_review(review, menu):
    mentioned_items = []
    for item in menu:
        if item in review:
            mentioned_items.append(item)
    return " ".join(mentioned_items)

In [11]:
df_exploded["mentioned_menu"] = df_exploded.apply(
    lambda row: find_menu_in_review(row["menu_sentence"], row["menu_name_split"]), axis=1
)

df_exploded

,rst_name,menu_sentence,menu_name_split,mentioned_menu
0,가장맛있는족발 구의역점,잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면서...,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어...",", 인, 알, 고"
1,가장맛있는족발 구의역점,매운 거 좋아 .,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어...",
2,가장맛있는족발 구의역점,아이들이랑 가서 반반 족발 먹었어요,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어...",", 발, 족발"
3,가장맛있는족발 구의역점,지점마다 서비스 내용이 다른가 봅니다,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어...",
4,가장맛있는족발 구의역점,일단 콩나물국 정말 맛있었어요,"[, 왕족, 알뜰, ), 불족발, 실속, 보쌈, 싱글족발, 1, 발, 싱글보쌈, 어...",
...,...,...,...,...
9225,황실짜장,마파두부덮밥 조금 맵지만 맛있어요 .,"[물만두, 만, 불짜장, 삼선해물탕, 삼선간짜장, 해물탕, 꽃, 울, 꽃빵튀김, 울...","만, 두"
9226,황실짜장,새우볶음밥 먹었는데 맛있었어요 .,"[물만두, 만, 불짜장, 삼선해물탕, 삼선간짜장, 해물탕, 꽃, 울, 꽃빵튀김, 울...",볶음
9227,황실짜장,쟁반짜장 맛있고 좋았습니다.,"[물만두, 만, 불짜장, 삼선해물탕, 삼선간짜장, 해물탕, 꽃, 울, 꽃빵튀김, 울...","쟁반짜장, 쟁반, 짜장"
9228,황실짜장,볶음밥 맛남ㅎ.,"[물만두, 만, 불짜장, 삼선해물탕, 삼선간짜장, 해물탕, 꽃, 울, 꽃빵튀김, 울...",볶음


In [ ]:
# 이 다음에 데이터프레임화 하면됨